In [1]:
import numpy as np
from astropy.table import Table
import astropy.table as astropy_table

In [2]:
table = Table.read('tables/photometry_HD_191939.fits')

In [3]:
# Assuming 'table' is your Astropy table and the first 5 columns are metadata, not photometric data.
criterion = 0.9  # Fraction of valid observations required

# Extract only the photometry columns (columns 6 onwards)
photometry_data = table.columns[5:]  # Assuming 0-based indexing, [5:] skips the first 5 columns

# Initialize an empty mask list for valid stars
valid_mask = []

# Loop through each photometric column (i.e., each star's data)
for col_name in photometry_data:
    col_data = table[col_name]  # Get the column data for the star

    # Calculate the fraction of valid (non-NaN) observations
    valid_fraction = np.sum(~np.isnan(col_data)) / len(col_data)
    
    # Check if the valid fraction meets the criterion
    if valid_fraction > criterion:
        valid_mask.append(True)
    else:
        valid_mask.append(False)

# The result is a mask indicating which columns (stars) meet the criterion
valid_columns_mask = np.array([False] * 5 + valid_mask)
# valid_columns_mask = np.array([True] * 5 + valid_mask)

# You can apply this mask to extract the valid stars' columns
# valid_star_columns = np.array(photometry_data)[valid_columns_mask]

In [4]:
valid_col = [col for i, col in enumerate(table.colnames) if valid_columns_mask[i]]

In [5]:
valid_table = table[valid_col]

In [24]:
relphot_table = Table(names=valid_table.colnames)

for row in valid_table:
    relphot_row = []
    row_sum = np.nansum(list(row))
    for i, value in enumerate(row):
        relphot_row.append(value / (row_sum - value))
    relphot_table.add_row(relphot_row)

/opt/anaconda3/envs/space/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:96: UserWarning: Warning: converting a masked element to nan.
  a = np.asanyarray(a)


In [25]:
# norm

relphot_norm_table = relphot_table.copy()

for colname in relphot_norm_table.colnames:
    # Get the maximum value in the column (ignoring NaNs)
    col_mean = np.nanmean(relphot_norm_table[colname])
    
    if col_mean != 0:  # Prevent division by zero
        # Normalize the column by dividing by its max value
        relphot_norm_table[colname] = relphot_norm_table[colname] / col_mean
    else:
        relphot_norm_table[colname] = np.nan

In [46]:
relphot_full_table = astropy_table.hstack([table[table.colnames[:5]], relphot_table])

In [50]:
relphot_full_table.write('tables/relphot/relphot_HD_191939.fits', overwrite=True)

In [47]:
gpb = relphot_full_table.group_by(['night', 'band'])
binned_relphot = gpb.groups.aggregate(np.median)

In [49]:
binned_relphot.write('tables/binned/binned_HD_191939.fits', overwrite=True)

In [19]:
# Step 1: Keep the first 5 columns as they are.
first_5_columns = table.columns[:5]

# Step 2: Apply the mask to the photometry columns
# The mask already includes both the first 5 columns and the photometric data columns.
masked_columns = table.columns[valid_columns_mask]

# Step 3: Rebuild the table by combining the first 5 columns with the valid photometry columns
valid_table = Table()

# Add the first 5 columns
for col_name in first_5_columns:
    valid_table[col_name] = table[col_name]

# Add the valid photometry columns
for col_name in masked_columns:
    valid_table[col_name] = table[col_name]

ValueError: Illegal type <class 'numpy.bool_'> for table item access